<a href="https://colab.research.google.com/github/SupplyAnalytics/Marketing_Team_Projects/blob/main/DP_Subcat_Duplicacy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
import requests
from io import BytesIO
from PIL import Image
from tqdm import tqdm
import numpy as np
from tensorflow import keras
from keras.applications import MobileNetV2
from keras.preprocessing.image import img_to_array
from sklearn.metrics.pairwise import cosine_similarity
from concurrent.futures import ThreadPoolExecutor

In [ ]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Read the Excel file into a DataFrame
csv_file_path = '/content/drive/My Drive/DP_TASK_40.csv'
df = pd.read_csv(csv_file_path)

In [ ]:
# Load MobileNetV2 without the top (classification) layers
base_model = MobileNetV2(weights='imagenet', include_top=False)
model = keras.Model(inputs=base_model.input, outputs=base_model.output)
# Function to extract feature vectors from image URLs
def extract_feature_vector(image_url):
    response = requests.get(image_url)
    if response.status_code == 200:
        image_data = BytesIO(response.content)
        img = Image.open(image_data)

        if img.format != 'JPEG':
            img = img.convert("RGB")
            image_data = BytesIO()
            img.save(image_data, format='JPEG')
            image_data.seek(0)

        img = img.resize((224, 224))
        img_array = img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = keras.applications.mobilenet_v2.preprocess_input(img_array)
        feature_vector = model.predict(img_array).flatten()
        return feature_vector
    else:
       return None

9406464/9406464 [==============================] - 0s 0us/step


In [ ]:
print(df.shape)
print(df.ndim)
print(df)

(6507, 3)
2
                                              App_Image  VariantId  \
0     https://bijnis.s3.amazonaws.com/PRODUCTION/upl...    2418801   
1     https://bijnis.s3.amazonaws.com/PRODUCTION/upl...    1960582   
2     https://bijnis.s3.amazonaws.com/PRODUCTION/upl...    2399469   
3     https://bijnis.s3.amazonaws.com/PRODUCTION/upl...    2263312   
4     https://bijnis.s3.amazonaws.com/PRODUCTION/upl...    2418802   
...                                                 ...        ...   
6502  https://bijnis.s3.amazonaws.com/PRODUCTION/upl...    2544165   
6503  https://bijnis.s3.amazonaws.com/PRODUCTION/upl...    2552536   
6504  https://bijnis.s3.amazonaws.com/PRODUCTION/upl...    2554748   
6505  https://bijnis.s3.amazonaws.com/PRODUCTION/upl...    2552730   
6506  https://shoekonnect.s3.amazonaws.com/PRODUCTIO...    1882197   

                      SubCategory  
0     Synthetic Handmade Slippers  
1     Synthetic Handmade Slippers  
2     Synthetic Handmade Slippers  
3  

In [ ]:
# Grouping the dataframe by subcategory
grouped = df.groupby('SubCategory')
result_dict = {}

def process_variant(row):
    variantid = row['VariantId']
    feature_vector = None
    try:
        feature_vector = extract_feature_vector(row['App_Image'])
    except Exception as e:
        print(f"Error extracting feature vector for variant {variantid}: {e}")
    return variantid, feature_vector



for SubCategory, group_df in grouped:
    subcategory_dict = {}
    with ThreadPoolExecutor(max_workers=80) as executor:
        futures = [executor.submit(process_variant, row) for index, row in group_df.iterrows()]
        for future in tqdm(futures, desc=f"Processing subcategory {SubCategory}"):
            variantid, feature_vector = future.result()
            if feature_vector is not None:
                subcategory_dict[variantid] = feature_vector


    result_dict[SubCategory] = subcategory_dict


print(result_dict)


Processing subcategory Synthetic Handmade Slippers:   0%|          | 0/6507 [00:00<?, ?it/s]

1/1 [==============================] - 57s 57s/step


1/1 [==============================] - 63s 63s/step


1/1 [==============================] - 55s 55s/step


Processing subcategory Synthetic Handmade Slippers:   0%|          | 1/6507 [00:52<94:10:27, 52.11s/it]

...

1/1 [==============================] - 5s 5s/step


Processing subcategory Synthetic Handmade Slippers:  66%|██████▌   | 4278/6507 [34:03<34:07,  1.09it/s]

1/1 [==============================] - 2s 2s/step
Error extracting feature vector for variant 2358597: 'Functional' object has no attribute 'predict_function'
1/1 [==============================] - 4s 4s/step
Error extracting feature vector for variant 2332045: 'Functional' object has no attribute 'predict_function'Error extracting feature vector for variant 2560870: 'Functional' object has no attribute 'predict_function'
Error extracting feature vector for variant 2528622: 'Functional' object has no attribute 'predict_function'
1/1 [==============================] - ETA: 0sError extracting feature vector for variant 2541965: 'Functional' object has no attribute 'predict_function'
Error extracting feature vector for variant 2524637: 'Functional' object has no attribute 'predict_function'
Error extracting feature vector for variant 2471979: 'Functional' object has no attribute 'predict_function'
1/1 [==============================] - 7s 7s/step
Error extracting feature vector for varian

Processing subcategory Synthetic Handmade Slippers: 100%|██████████| 6507/6507 [45:15<00:00,  2.40it/s]   


{'Synthetic Handmade Slippers': {2418801: array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), 1960582: array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), 2399469: array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), 2263312: array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), 2418802: array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), 2418769: array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), 2418805: array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), 1960554: array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), 2326743: array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), 2386075: array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), 2359827: array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), 2354385: array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), 2081076: array([0.        , 0.21516037, 0.        , ..., 0.        , 0.        ,
       0.        ], dtype=float32), 2394040: array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), 2364122: array([0., 0., 0., ..., 0., 0., 0.]

In [ ]:
# List to store duplicate variants
duplicate_variants = []

# Iterate over subcategories in result_dict with tqdm
for subcategory, inner_dict in  result_dict.items():
    variant_ids = list(inner_dict.keys())
    print(f'Length of variant ids for {subcategory}: {len(variant_ids)}')

    for i in range(len(variant_ids)):
        for j in range(i+1, len(variant_ids)):
            similarity = cosine_similarity([inner_dict[variant_ids[i]]], [inner_dict[variant_ids[j]]])[0][0]
            print(f'Comparing {subcategory}, {i}, {j}: {similarity}')

            if similarity >= 0.90:
                duplicate_variants.append({
                    'VariantId1': variant_ids[i],
                    'VariantId2': variant_ids[j],
                    'Similarity': similarity})


Streaming output truncated to the last 5000 lines.
Comparing Synthetic Handmade Slippers, 2162, 5074: 0.24789151549339294
Comparing Synthetic Handmade Slippers, 2162, 5075: 0.3430481553077698
Comparing Synthetic Handmade Slippers, 2162, 5076: 0.28566116094589233
Comparing Synthetic Handmade Slippers, 2162, 5077: 0.3228148818016052
Comparing Synthetic Handmade Slippers, 2162, 5078: 0.3662126958370209
Comparing Synthetic Handmade Slippers, 2162, 5079: 0.34197789430618286
Comparing Synthetic Handmade Slippers, 2162, 5080: 0.2831341028213501
Comparing Synthetic Handmade Slippers, 2162, 5081: 0.2608364224433899
Comparing Synthetic Handmade Slippers, 2162, 5082: 0.3655001223087311
Comparing Synthetic Handmade Slippers, 2162, 5083: 0.3155112862586975
Comparing Synthetic Handmade Slippers, 2162, 5084: 0.3497282564640045
Comparing Synthetic Handmade Slippers, 2162, 5085: 0.2446765899658203
Comparing Synthetic Handmade Slippers, 2162, 5086: 0.260761559009552
Comparing Synthetic Handmade Slippers

In [ ]:

# Print number of duplicate variants found
print(f"Number of duplicate variants found: {len(duplicate_variants)}")

In [ ]:
# Create a DataFrame from the list of duplicate variants
duplicate_df = pd.DataFrame(duplicate_variants)

# Save the duplicate variants to a CSV file
output_csv_path = '/content/drive/My Drive/DP_TASK_40_Result.csv'

duplicate_df.to_csv(output_csv_path, index=False)

print(f"Duplicate variants saved to: {output_csv_path}")